In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import skimage.io

import os.path

import sys
sys.path.insert(0, '/home/jr0th/github/segmentation/code/')
import helper.metrics

import numpy as np

Using TensorFlow backend.


In [2]:
dl_label = '/home/jr0th/github/segmentation/experiments/BBBC022_hand/DL_labels/'
gt_label = '/home/jr0th/github/segmentation/experiments/BBBC022_hand/GT_labels/'

mat_dl_dir = '/home/jr0th/github/segmentation/experiments/BBBC022_hand/IoU_DL/'

In [3]:
labels_dl = skimage.io.imread_collection(dl_label + "*.png")

# buffer for results
results = np.empty(shape = (0, 4))

# loop over all images
for index in range(len(labels_dl)):
    
    # print progress
    print(index + 1, '/', len(labels_dl))

    path = labels_dl.files[index]
    filename = os.path.basename(path)
    filename_wo_ext = os.path.splitext(filename)[0]
    
    # get labels
    label_dl = labels_dl[index]
    label_gt = skimage.io.imread(gt_label + filename)

    nb_gt_cells = np.max(label_gt)
    
    [nb_overdetection, nb_underdetection, mean_IoU, IoUs] = helper.metrics.compare_two_labels(label_dl, label_gt, True)
    
    np.save(mat_dl_dir + filename_wo_ext, IoUs)
    
    result = np.hstack((nb_overdetection, nb_underdetection, mean_IoU, nb_gt_cells))
    results = np.vstack((results, result))

print(results)

# calculate error ratio
sums = np.sum(results, axis = 0)
nb_gt_cells_total = sums[3]
nb_errors = sums[0] + sums[1]

err_ratio = nb_errors / nb_gt_cells_total
print(err_ratio)

np.savetxt('./results_table_DL.txt', results)
np.savetxt('./results_summary_DL.txt', np.array([nb_errors, nb_gt_cells_total, err_ratio]))

1 / 99
2 / 99
3 / 99
4 / 99
5 / 99
6 / 99
7 / 99
8 / 99
9 / 99
10 / 99
11 / 99
12 / 99
13 / 99
14 / 99
15 / 99
16 / 99
17 / 99
18 / 99
19 / 99
20 / 99
21 / 99
22 / 99
23 / 99
24 / 99
25 / 99
26 / 99
27 / 99
28 / 99
29 / 99
30 / 99
31 / 99
32 / 99
33 / 99
34 / 99
35 / 99
36 / 99
37 / 99
38 / 99
39 / 99
40 / 99
41 / 99
42 / 99
43 / 99
44 / 99
45 / 99
46 / 99
47 / 99
48 / 99
49 / 99
50 / 99
51 / 99
52 / 99
53 / 99
54 / 99
55 / 99
56 / 99
57 / 99
58 / 99
59 / 99
60 / 99
61 / 99
62 / 99
63 / 99
64 / 99
65 / 99
66 / 99
67 / 99
68 / 99
69 / 99
70 / 99
71 / 99
72 / 99
73 / 99
74 / 99
75 / 99
76 / 99
77 / 99
78 / 99
79 / 99
80 / 99
81 / 99
82 / 99
83 / 99
84 / 99
85 / 99
86 / 99
87 / 99
88 / 99
89 / 99
90 / 99
91 / 99
92 / 99
93 / 99
94 / 99
95 / 99
96 / 99
97 / 99
98 / 99
99 / 99
[[   3.           14.            0.74736208   91.        ]
 [   1.           17.            0.66335988   99.        ]
 [   2.           15.            0.67529184   76.        ]
 [   1.           16.            0.76823